## Query header

In this notebook, we show several ways to query the fits headers with Gen3\
Craig Lage - 03-Jun-21

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/LATISS', collections="LATISS/raw/all")

In [ ]:
butler = Butler('/repo/LATISS', collections="LATISS/runs/quickLook")

In [ ]:
butler = Butler('/repo/LSSTComCam', collections="LSSTComCam/raw/all")

In [ ]:
dayObs = 20220609
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter, record.target_name)


In [ ]:
exposureList[:][0]

In [ ]:
exposureList[0][1].fields

In [ ]:
from lsst.daf.butler import Butler


nTot=0
for [expId,record] in exposureList:
    nExp = 0
    for detector in range(9):
        try:
            mData = butler.get('raw.metadata', detector=detector, exposure=expId)
        except:
            nTot += 1
            nExp += 1
            #print(f"Detector {detector} from expId {expId} missing")
    if nExp != 0:
        print(f" expId {expId} is missing {nExp} files.")
print(f" Total of {n} exposures missing")

In [ ]:
expId = 2022050300200
mData = butler.get('raw.metadata', detector=0, exposure=expId)
for key in mData.keys():
    print(key, mData[key])

In [ ]:
expId = 2022040600654
exp = butler.get('quickLookExp', detector=0, exposure=expId)
